In [1]:
# @title Installation der Softwarepakete

%%capture
#Install libraries
!pip install stable_diffusion_videos

import torch

from stable_diffusion_videos import StableDiffusionWalkPipeline

#Prepare the pipeline
pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
).to("cuda")

#Videocodecs and preferences
from IPython.display import HTML
from base64 import b64encode

def visualize_video_colab(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

## Videogenerierung

In der nächsten Codezelle kannst du ein Animationsvideo generieren.
Beschreibe dazu mit "Prompt_1" das Startbild und mit "Prompt_2" das Endbild.
Das Modell generiert dann beliebig viele Bilder (frames per second - fps) pro Sekunde und fügt diese zu einem Video zusammen.

In [2]:
# @title Generiere ein Animationsvideo nach deiner Beschreibung!

#Default parameters
Länge = 5 # @param {type:"slider", min:4, max:16, step:1}
Prompt_1 = "tree" # @param {type:"string"}
Prompt_2 = "building" # @param {type:"string"}
fps = 5 # @param {type:"slider", min:1, max:10, step:1}

#Generate and show a video
video_path = pipeline.walk(
    [Prompt_1, Prompt_2],
    [42, 321],
    fps=fps,                      # use 5 for testing, 25 or 30 for better quality
    num_interpolation_steps=Länge,  # use 3-5 for testing, 30 or more for better results
    height=512,                 # use multiples of 64 if > 512. Multiples of 8 if < 512.
    width=512,                  # use multiples of 64 if > 512. Multiples of 8 if < 512.
    guidance_scale=6.5,         # Higher adheres to prompt more, lower lets model take the wheel
    num_inference_steps=30,     # Number of diffusion steps per image generated. 50 is good default
)
visualize_video_colab(video_path)

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

## Musikvideogenerierung

In der nächsten Codezelle kannst du ein Animationsvideo generieren, welches sich an einen Ausschnitt von einem dir vorgegebenen Song anpasst.
Gib dazu den Start- und Endpunkt des Songs in Sekunden an und beschreibe mit "Prompt_1" wieder das Startbild und mit "Prompt_2" das Endbild.
Das Modell generiert dann beliebig viele Bilder (frames per second - fps) pro Sekunde und fügt diese zu einem Video zusammen.

### Hinweis
-------
Lade ein Musikstück deiner Wahl in Colab hoch [1] und gib unter "Song" den vollständigen Dateinamen an.

[1] Das geht, indem du links auf das Symbol für Dateien/Ordnerverzeichnisse klickst und eine Datei in den Reiter links hineinziehst oder auf das "Dateiupload"-Symbol in den Reiter links oben klickst.

In [ ]:
# @title Generiere ein Animationsvideo nach deiner Beschreibung!

#Default parameters
Prompt_1 = "blueberry spaghetti" # @param {type:"string"}
Prompt_2 = "strawberry spaghetti" # @param {type:"string"}
fps = 10 # @param {type:"slider", min:1, max:10, step:1}
Start = 1 # @param {type:"number"}
Ende = 9 # @param {type:"number"}
Song = "" # @param {type:"string"}

# Display song if needed
# from IPython.display import Audio
# Audio(filename=Song)

# Seconds in the song
audio_offsets = [Start, Ende]

# Convert seconds to frames
num_interpolation_steps = [(b-a) * fps for a, b in zip(audio_offsets, audio_offsets[1:])]

#Generate and show a video
video_path = pipeline.walk(
    prompts=[Prompt_1, Prompt_2],
    seeds=[42, 321],
    num_interpolation_steps=num_interpolation_steps,
    height=512,                            # use multiples of 64
    width=512,                             # use multiples of 64
    audio_filepath=Song,    # Use your own file
    audio_start_sec=audio_offsets[0],       # Start second of the provided audio
    fps=fps,                               # important to set yourself based on the num_interpolation_steps you defined
    batch_size=4,                          # increase until you go out of memory.
    output_dir='./dreams',                 # Where images will be saved
    name=None,                             # Subdir of output dir. will be timestamp by default
)
visualize_video_colab(video_path)